In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
from src.inference import get_feature_store

In [4]:
from datetime import datetime, timedelta
import pandas as pd  

# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),
)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]

2025-03-03 17:58:12,385 INFO: Initializing external client
2025-03-03 17:58:12,385 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 17:58:14,357 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214680
Fetching data from 2025-02-02 22:58:12.359616+00:00 to 2025-03-03 21:58:12.359616+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (5.89s) 


In [5]:
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)

,pickup_hour,pickup_location_id,rides
0,2025-02-03 18:00:00+00:00,2,0
1,2025-02-03 19:00:00+00:00,2,0
2,2025-02-03 20:00:00+00:00,2,0
3,2025-02-03 21:00:00+00:00,2,0
4,2025-02-03 22:00:00+00:00,2,0
...,...,...,...
169671,2025-03-03 17:00:00+00:00,263,114
169672,2025-03-03 18:00:00+00:00,263,116
169673,2025-03-03 19:00:00+00:00,263,102
169674,2025-03-03 20:00:00+00:00,263,87


In [6]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 169676 entries, 0 to 169926
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype                  
---  ------              --------------   -----                  
 0   pickup_hour         169676 non-null  datetime64[us, Etc/UTC]
 1   pickup_location_id  169676 non-null  int32                  
 2   rides               169676 non-null  int32                  
dtypes: datetime64[us, Etc/UTC](1), int32(2)
memory usage: 3.9 MB


In [7]:
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

In [8]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 169676 entries, 0 to 169926
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   pickup_hour         169676 non-null  datetime64[us]
 1   pickup_location_id  169676 non-null  int32         
 2   rides               169676 non-null  int32         
dtypes: datetime64[us](1), int32(2)
memory usage: 3.9 MB


In [10]:
from src.data_utils import transform_ts_data_info_features_and_target
features = transform_ts_data_info_features_and_target(ts_data, window_size=24*28, step_size=23)

In [19]:
features, targets = features

In [22]:
from src.inference import load_batch_of_features_from_store
current_date = pd.Timestamp.now(tz='Etc/UTC')
features = load_batch_of_features_from_store(current_date)

2025-03-03 18:15:48,738 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-03 18:15:48,750 INFO: Initializing external client
2025-03-03 18:15:48,750 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 18:15:49,820 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214680
Fetching data from 2025-02-02 23:15:48.738586+00:00 to 2025-03-03 22:15:48.738586+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.96s) 


In [23]:
current_date

Timestamp('2025-03-03 23:15:48.738586+0000', tz='Etc/UTC')

In [24]:
features, targets = features

In [25]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_hour,pickup_location_id
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2025-03-03 18:00:00,2
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2025-03-03 18:00:00,3
2,0,1,3,3,6,2,3,6,2,0,...,3,5,1,2,2,3,0,1,2025-03-03 18:00:00,4
3,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,2025-03-03 18:00:00,6
4,1,5,1,2,0,0,0,0,1,0,...,3,2,8,3,4,1,3,4,2025-03-03 18:00:00,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,2025-03-03 18:00:00,259
247,1,3,0,1,0,0,0,0,0,0,...,0,1,4,3,2,5,1,6,2025-03-03 18:00:00,260
248,41,24,17,16,11,3,1,3,0,0,...,7,16,25,36,32,26,33,43,2025-03-03 18:00:00,261
249,73,43,34,22,4,6,1,3,1,1,...,118,98,79,71,82,97,87,82,2025-03-03 18:00:00,262


In [17]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-03-03 18:11:22,695 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-03 18:11:22,717 INFO: Initializing external client
2025-03-03 18:11:22,721 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 18:11:23,850 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214680


In [26]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

In [27]:
predictions

,pickup_location_id,predicted_demand
0,2,0.0
1,3,0.0
2,4,1.0
3,6,0.0
4,7,3.0
...,...,...
246,259,0.0
247,260,3.0
248,261,37.0
249,262,69.0


In [28]:
predictions.sort_values("predicted_demand", ascending=False).head(10)["pickup_location_id"].values

array([161, 237, 236, 162, 132, 163, 230, 142, 138, 239])